In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pip install -q --upgrade bitsandbytes transformers accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

before loading torch, we run the cell below to enable dynamic memory segment expansion (that PyTorch can grow its memory blocks on the GPU as needed, instead of pre-allocating fixed chunks), which helps prevent out-of-memory (OOM) crashes due to memory fragmentation when using GPU-heavy models like LLaMA.

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# Imports
import torch, json, re, gc
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import pandas as pd
from tqdm import tqdm
from huggingface_hub import login
login(token="", add_to_git_credential=True)

## Prompts

In [ ]:
SYSTEM_PROMPT = (
    "You are a precise JSON generator for mathematical QA datasets. "
    "Your ONLY valid output is a JSON object with exactly one key \"questions\" "
    "whose value is a list containing one and only one string. "
    "NEVER add markdown fences, labels, explanations, or additional keys. "
    "NEVER prefix your output with any words like “Question:” or “Sure! Here is”. "
    "Your output MUST start with “{” and end with “}”. "
    "ALWAYS verify your output is valid JSON before replying."
)

USER_TEMPLATE = (
    "Task:\n"
    "1. Read the theorem below.\n"
    "2. Write ONE question that could be answered by citing that theorem.\n"
    "   • Use clear, graduate-level mathematical language.\n"
    "   • Include LaTeX for any symbols or formulas.\n"
    "3. Return your answer as a JSON object with exactly one key \"questions\" "
    "whose value is a list containing exactly one string.\n\n"
    "Theorem:\n"
    "{THEOREM_TEXT}\n"
)

SYSTEM_PROMPT_alt = (
    "You are a formatting guard.\n"
    "Your ONLY valid reply is a JSON object that:\n"
    "• has exactly one key named \"questions\"\n"
    "• whose value is a JSON list containing exactly one string\n\n"
    "Never add any other keys, text, Markdown fences, or prefixes.\n"
    "Your reply MUST begin with “{” and end with “}”.\n"
    "Always verify the JSON is syntactically valid before sending."
)

USER_TEMPLATE_alt = (
    "Task:\n"
    "You are building a retrieval-augmented QA system for graduate-level mathematics.\n"
    "For the theorem given below, create ONE natural-language question that a student\n"
    "would ask if they needed to cite (or apply) this result while solving a problem.\n\n"
    "Guidelines:\n"
    "•  Use precise, graduate-level wording; include LaTeX for symbols where helpful.\n"
    "•  Do NOT mention equation numbers, labels, “Theorem X”, “Lemma Y”, or citations.\n"
    "•  Do NOT add explanatory sentences, bullet points, or multiple questions.\n"
    "•  Output EXACTLY one JSON object with key \"questions\" whose value is a list\n"
    "  containing exactly ONE string — your question.\n\n"
    "Return ONLY the JSON object. No extra prose.\n\n"
    "Theorem:\n"
    "{THEOREM_TEXT}"
)

## Load Model

In [ ]:
# 1) Load tokenizer & model
MODEL_ID = "meta-llama/Llama-3.2-3B-Instruct"

# 1) Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left" ###

# 2) 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# 3) Model loading
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    low_cpu_mem_usage=True         # offloads some weights to CPU if needed
)

model.config.pad_token_id = tokenizer.eos_token_id
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,)

## Main function

In [ ]:
def generate_questions_batch(df, system_prompt, user_template,
                             tokenizer, model,
                             batch_size=4, max_new_tokens=120):
    """
    df: DataFrame with column 'content'
    tokenizer: HuggingFace AutoTokenizer
    model:     HuggingFace AutoModelForCausalLM (already on GPU)
    """
    prompts = [
        f"<<SYS>>\n{system_prompt}\n<</SYS>>\n\n"
        f"[INST]\n{user_template.format(THEOREM_TEXT=theo)}[/INST]"
        for theo in df["content"]
    ]

    questions = []
    device = model.device
    total_batches = (len(prompts) + batch_size - 1) // batch_size

    for i in tqdm(range(0, len(prompts), batch_size),
                  desc="Generating questions",
                  total=total_batches,
                  unit="batch"):
        batch_prompts = prompts[i : i + batch_size]

        # 1) Tokenize the whole batch in one go
        inputs = tokenizer(
            batch_prompts,
            padding=True,
            truncation=True,
            return_tensors="pt"
        ).to(device)

        # 2) Generate in one forward pass
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                top_p=1.0,
                eos_token_id=tokenizer.eos_token_id
            )

        # 1. real prompt length per row  (tensor shape: [batch])
        prompt_lens = inputs["attention_mask"].sum(dim=1)

        # 2. cut away the prompt for every row without a Python loop
        gen_ids = [
            seq[p_len:]            # keep only the new tokens
            for seq, p_len in zip(output_ids, prompt_lens.tolist())
        ]
        # 3b) Decode only those new tokens
        batch_texts = tokenizer.batch_decode(
            gen_ids,
            skip_special_tokens=True
        )

        # 4) Parse each output exactly like your single-call version
        for out_text in batch_texts:
            # JSON‐block + backslash‐escape
            #print(f"\n out text: {out_text}\n")
            match = re.search(
                r'\{[^{}]*"questions"\s*:\s*\[\s*".+?"\s*\][^{}]*\}',
                out_text,
                flags=re.DOTALL
            )
            if match:
                js_blob = match.group(0).replace("\\", "\\\\")
                #print(f"\n\n Match: {js_blob}\n\n")
                try:
                    js = json.loads(js_blob)
                    questions.append(js["questions"][0])
                    continue
                except:
                    pass

            # Fallback
            for line in out_text.splitlines():
                clean = line.strip()
                if clean and not clean.lower().startswith("question"):
                    questions.append(clean)
                    break
            else:
                questions.append("__FAILED__")

    df["question"] = questions
    return df

## Single Execution

In [ ]:
INPUT_CSV  = "/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/theorems_and_lemmas_200.csv"
INPUT_CSV = "/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/failed_passages_I.csv"

INPUT_CSV = "/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/passages_200.csv"
OUTPUT_CSV = "/content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_v7.csv"

#  Load data
df = pd.read_csv(INPUT_CSV)

#  Run
df = generate_questions_batch(
    df,
    system_prompt=SYSTEM_PROMPT,
    user_template=USER_TEMPLATE,
    tokenizer=tokenizer,
    model=model,
    batch_size=32,
    max_new_tokens=100
)

df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved to {OUTPUT_CSV}")

In [ ]:
# clean up cache for less gpu memory usage
torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

31

# Generator for Big Dataframe

In [ ]:
def run_generation_with_checkpoints(
    df_full,
    chunk_size,
    checkpoint_path,
    already_done=0,
    **gen_kwargs
):
    """
    Processes the full DataFrame in chunks and saves to disk after each chunk.
    Resumes from existing checkpoint if present.

    After each chunk, synchronizes GPU, empties cache, and runs GC to avoid fragmentation/OOM.
    """
    total_rows = len(df_full)

    # Load existing checkpoint if present
    if os.path.exists(checkpoint_path):
        processed_df = pd.read_csv(checkpoint_path)
        already_done = len(processed_df)
        print(f"Resuming from checkpoint. Already processed {already_done} rows.")
    else:
        processed_df = pd.DataFrame()  # empty

    # Iterate over chunks
    for start in range(already_done, total_rows, chunk_size):
        end = min(start + chunk_size, total_rows)
        chunk_df = df_full.iloc[start:end].copy()
        chunk_df.reset_index(drop=True, inplace=True)
        chunk_df['id'] = chunk_df.index

        print(f"\nProcessing rows {start} to {end - 1}")
        # Run generation on the chunk
        chunk_with_qs = generate_questions_batch(chunk_df, **gen_kwargs)

        # Append results
        processed_df = pd.concat([processed_df, chunk_with_qs], ignore_index=True)

        # Save checkpoint without writing the index column
        processed_df.to_csv(checkpoint_path, index=False)
        print(f"Saved checkpoint with {len(processed_df)} rows to {checkpoint_path}")

        # Clear GPU cache and Python garbage to avoid fragmentation
        torch.cuda.synchronize()
        torch.cuda.empty_cache()
        gc.collect()
        print("Cleared CUDA cache and ran garbage collection.")

    return processed_df

In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/df_100_chunk_1.csv")
df.head()

,id,paper id,title,categories,type,content
0,0,2501.00724,category o for quantum loop algebras,math.rt math.qa,theorems,\label{thm:main}\n\t\nConsider any Kac-Moody L...
1,1,2501.00724,category o for quantum loop algebras,math.rt math.qa,theorems,\label{thm:my refined}\n\nLet $\fg$ be of fini...
2,2,2501.00724,category o for quantum loop algebras,math.rt math.qa,theorems,\label{thm:toroidal}\n\nFor a polynomial $\ell...
3,3,2501.00724,category o for quantum loop algebras,math.rt math.qa,theorems,\label{thm:simple}\n\n(\cite{HJ}) Up to isomor...
4,4,2501.00724,category o for quantum loop algebras,math.rt math.qa,theorems,\label{thm:quantum to shuffle}\n\nWe have $\em...


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/df_100_chunk_1.csv")

checkpoint_path="/content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_1.csv"

final_df = run_generation_with_checkpoints(
    df_full=df,
    chunk_size=1000,
    checkpoint_path=checkpoint_path,
    system_prompt=SYSTEM_PROMPT,
    user_template=USER_TEMPLATE,
    tokenizer=tokenizer,
    model=model,
    batch_size=32,
    max_new_tokens=100
)

Resuming from checkpoint. Already processed 6000 rows.

Processing rows 6000 to 6999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [17:21<00:00, 32.54s/batch]


Saved checkpoint with 7000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_1.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 7000 to 7999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [17:02<00:00, 31.96s/batch]


Saved checkpoint with 8000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_1.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 8000 to 8999


...

Saved checkpoint with 15000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_1.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 15000 to 15999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [16:30<00:00, 30.95s/batch]


Saved checkpoint with 16000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_1.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 16000 to 16666


Generating questions:   0%|          | 0/21 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 21/21 [11:37<00:00, 33.21s/batch]


Saved checkpoint with 16667 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_1.csv
Cleared CUDA cache and ran garbage collection.


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/df_100_chunk_2.csv")

checkpoint_path="/content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_2.csv"

final_df = run_generation_with_checkpoints(
    df_full=df,
    chunk_size=1000,
    checkpoint_path=checkpoint_path,
    system_prompt=SYSTEM_PROMPT,
    user_template=USER_TEMPLATE,
    tokenizer=tokenizer,
    model=model,
    batch_size=32,
    max_new_tokens=100
)

Resuming from checkpoint. Already processed 14000 rows.

Processing rows 14000 to 14999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [17:11<00:00, 32.25s/batch]


Saved checkpoint with 15000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_2.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 15000 to 15999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [17:38<00:00, 33.07s/batch]


Saved checkpoint with 16000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_2.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 16000 to 16666


Generating questions:   0%|          | 0/21 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 21/21 [12:40<00:00, 36.23s/batch]


Saved checkpoint with 16667 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_2.csv
Cleared CUDA cache and ran garbage collection.


In [ ]:
df = pd.read_csv("/content/drive/Shareddrives/Master_Thesis/Query_Creation/Data/df_100_chunk_3.csv")

checkpoint_path="/content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_3.csv"

final_df = run_generation_with_checkpoints(
    df_full=df,
    chunk_size=1000,
    checkpoint_path=checkpoint_path,
    system_prompt=SYSTEM_PROMPT,
    user_template=USER_TEMPLATE,
    tokenizer=tokenizer,
    model=model,
    batch_size=32,
    max_new_tokens=100
)

Resuming from checkpoint. Already processed 10000 rows.

Processing rows 10000 to 10999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [16:23<00:00, 30.75s/batch]


Saved checkpoint with 11000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_3.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 11000 to 11999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [17:36<00:00, 33.03s/batch]


Saved checkpoint with 12000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_3.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 12000 to 12999


...

Saved checkpoint with 15000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_3.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 15000 to 15999


Generating questions:   0%|          | 0/32 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 32/32 [18:42<00:00, 35.07s/batch]


Saved checkpoint with 16000 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_3.csv
Cleared CUDA cache and ran garbage collection.

Processing rows 16000 to 16666


Generating questions:   0%|          | 0/21 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Generating questions: 100%|██████████| 21/21 [11:38<00:00, 33.26s/batch]


Saved checkpoint with 16667 rows to /content/drive/Shareddrives/Master_Thesis/Query_Creation/Results/queries_chunk_3.csv
Cleared CUDA cache and ran garbage collection.
